In [1]:
import requests
import pandas as pd
import numpy as np
import json
import sys

requests.packages.urllib3.disable_warnings()

In [2]:
api_multi = 'https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/multiResult?interfaceID=MultiSRCInformation'
api_multi_filter = "https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/multiResult?interfaceID=MultiSRCInformation&$filter=parentStudyID eq '20M0082'"

In [3]:
session = requests.Session()
session.auth = (u<'ID'>, <'PWD'>)
session.verify = False

In [4]:
response = session.get(api_multi)

In [6]:
def json_flatten(json_dict):

    def inner_flatten(x, name=''):
        single_dict = {}
        if isinstance(x, dict):
            flattened_dict = {}
            for key,value in x.items():
                #recursively calling on deeper layers of the json
                flattened_x = inner_flatten(value, name + key + '_')
                flattened_dict.update(flattened_x)
            single_dict.update(flattened_dict)
                    
        elif isinstance(x, list):
            flattened_dict = {}
            flattened_x = []
            for obj in x:
                flattened_obj = inner_flatten(obj, name)
                flattened_x.append(flattened_obj)

            #will always be true due to the nature of the inner function return type
            if all(isinstance(y,dict) for y in flattened_x):
                for i, d in enumerate(flattened_x):
                    for k, v in d.items():
                        flattened_dict[f'{k}_{i+1}'] = v
                    
            single_dict.update(flattened_dict)
            
        else:
            single_dict[name[:-1]] = x
            
        return single_dict

    return inner_flatten(json_dict)

In [5]:
IDs = set()
parentIDs = set()
multi_json = response.json()

In [7]:
for submission in multi_json:
    parentStudyID = submission['parentStudyID']
    srcID = submission['ID']

    parentIDs.add(parentStudyID)
    IDs.add(srcID)    

In [8]:
for json in multi_json:
    parentStudyID = json['parentStudyID']
    srcID = json['ID']

    parentIDs.add(parentStudyID)
    IDs.add(srcID)

In [9]:
#loop through submissionIDs and perform single RS api call on them. insert json into singleRSC table
SRCSingleAPI = "https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/singleResult?interfaceID=ScientificReviewSubmission&resourceID="

singleSRC_data = list()
for id in IDs:
    response = session.get(SRCSingleAPI + '\'' + id + '\'')
    singleSRC_data.append(json_flatten(response.json()['data']))

singleSRC_table = pd.DataFrame(singleSRC_data)

In [10]:
#loop through parentIDs and perform multi RS api call on them. insert json into multiRSC table
SRCMultiAPI = "https://protect-training.cc.nih.gov/TRAINING-IRB/sd/PublicCustomLayouts/PSLib/WebApi/multiResult?interfaceID=MultiSRCInformation&$filter=parentStudyID eq "

multiSRC_data = list()
for id in parentIDs:
    response = session.get(SRCMultiAPI + '\'' + id + '\'')
    multiSRC_data = multiSRC_data + response.json()

multiSRC_table = pd.DataFrame(multiSRC_data)

In [ ]:
#Pushing data to postgresql

In [18]:
import psycopg2
from sqlalchemy import create_engine

In [19]:
db = create_engine('postgresql://<ID>:<PWD>@10.157.90.23:5432/PROTECT')
conn = db.connect()

In [47]:
multiSRC_table.to_sql('ScientificReviewStudyDetails', con=conn, if_exists='replace', index=False)

420

In [48]:
singleSRC_table.to_sql('ScientificReviewSubmissionDetails', con=conn, if_exists='replace', index=False)

34